In [1]:
import numpy as np

In [5]:
class DecisionTree:
    def __init__(self):
        self.tree = None

    def entropy_function(self, p):
        return -p * np.log2(p) if p > 0 else 0

    def calc_info_gain(self, data, classes, feature):
        nData = len(data)

        # Compute entropy before the split
        classValues, classCounts = np.unique(classes, return_counts=True)
        entropy_before = sum(self.entropy_function(classCounts[i] / nData) for i in range(len(classValues)))

        # Get unique values of the feature
        values, featureCounts = np.unique([datapoint[feature] for datapoint in data], return_counts=True)

        # Compute weighted entropy after split
        weighted_entropy = 0
        for i, value in enumerate(values):
            newClasses = [classes[idx] for idx, datapoint in enumerate(data) if datapoint[feature] == value]
            subClassValues, subClassCounts = np.unique(newClasses, return_counts=True)
            sub_entropy = sum(self.entropy_function(subClassCounts[j] / sum(subClassCounts)) for j in range(len(subClassValues)))
            weighted_entropy += (featureCounts[i] / nData) * sub_entropy

        return entropy_before - weighted_entropy

    def build_tree(self, data, classes, features):
        # If all classes are the same, return that class
        if len(set(classes)) == 1:
            return classes[0]

        # If no features left, return the majority class
        if not features:
            return max(set(classes), key=classes.count)

        # Calculate Information Gain for each feature
        info_gains = {feature: self.calc_info_gain(data, classes, feature) for feature in features}

        # Find the best feature to split on
        best_feature = max(info_gains, key=info_gains.get)

        # Create the tree node
        tree = {best_feature: {}}

        # Get unique values of the best feature
        unique_values = set(datapoint[best_feature] for datapoint in data)

        # Split data and recursively build branches
        for value in unique_values:
            subset_data = [datapoint for datapoint in data if datapoint[best_feature] == value]
            subset_classes = [classes[idx] for idx, datapoint in enumerate(data) if datapoint[best_feature] == value]
            remaining_features = [f for f in features if f != best_feature]
            tree[best_feature][value] = self.build_tree(subset_data, subset_classes, remaining_features)

        return tree

    def fit(self, data, classes, features):
        self.tree = self.build_tree(data, classes, features)

    def print_tree(self, tree=None, indent=""):
        if tree is None:
            tree = self.tree

        if isinstance(tree, dict):
            for key, value in tree.items():
                print(indent + str(key))
                self.print_tree(value, indent + "  ")
        else:
            print(indent + "→ " + str(tree))

# Dataset
data = [
    {"Deadline": "Urgent", "Party": "Yes", "Lazy": "Yes"},
    {"Deadline": "Urgent", "Party": "No", "Lazy": "Yes"},
    {"Deadline": "Near", "Party": "Yes", "Lazy": "Yes"},
    {"Deadline": "None", "Party": "Yes", "Lazy": "No"},
    {"Deadline": "None", "Party": "No", "Lazy": "Yes"},
    {"Deadline": "None", "Party": "Yes", "Lazy": "No"},
    {"Deadline": "Near", "Party": "No", "Lazy": "No"},
    {"Deadline": "Near", "Party": "No", "Lazy": "Yes"},
    {"Deadline": "Urgent", "Party": "Yes", "Lazy": "Yes"},
    {"Deadline": "Urgent", "Party": "No", "Lazy": "No"},
]

classes = ["Party", "Study", "Party", "Party", "Pub", "Party", "Study", "TV", "Party", "Study"]
features = ["Deadline", "Party", "Lazy"]

# Train decision tree
dt = DecisionTree()
dt.fit(data, classes, features)

dt.print_tree()


Party
  No
    Deadline
      Near
        Lazy
          No
            → Study
          Yes
            → TV
      Urgent
        → Study
      None
        → Pub
  Yes
    → Party
